In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.widgets import Slider

In [ ]:
def getCoords(items):
  for key, value in items:
    if key.startswith('x'):
      x = value
    if key.startswith('y'):
      y = value
    if key.startswith('z'):
      z = value
  return x,y,z

def norm(x, y, z):
  return math.sqrt(x**2 + y**2 + z**2)

# get from dict: https://stackoverflow.com/questions/10795973/python-dictionary-search-values-for-keys-using-regular-expression#10796073
def dist(row1, row2):
  x,y,z = getCoords(row1.iteritems())
  x2,y2,z2 = getCoords(row2.iteritems())
  return (x-x2), (y-y2), (z-z2)

def getKwgsCoords(**kwgs):
  return getCoords(kwgs.items())

In [ ]:
# TODO Partially complete
# Way of getting pairwise difference between every robot i and j
def getSeparations(df):
    robot_separations = {}
    i = 0
    j = 0
    for group_name_i, df_group_i in df.groupby(np.arange(len(df.columns)) // 3, axis=1):
        print("iteration i: ", i)
        robot_separations[i] = {}
        j = 0
        for group_name_j, df_group_j in df.groupby(np.arange(len(df.columns)) // 3, axis=1):
            if i != j:
                print("iteration j: ", j)
                robot_separations[i][j] = {}
                for row_index_i, row_i in df_group_i.iterrows():
                    for row_index_j, row_j in df_group_j.iterrows():
                        print("rows: ", row_i, row_j)
                        x,y,z = dist(row_i, row_j)
                        robot_separations[i][j]['x'] = x
                        robot_separations[i][j]['y'] = y
                        robot_separations[i][j]['z'] = z
            j = j+1
            # print(row_index, '\n', " here is one row ", row, " end of row.")
        i = i+1
    return robot_separations

In [ ]:
data = pd.read_csv (r'run_300positions.csv')   
df = pd.DataFrame(data)
df.drop('time', axis=1, inplace=True)
df.drop(df.filter(regex='robotId').columns, axis=1, inplace=True)

# Take every 3 cols (xyz) and combine into tuples (https://stackoverflow.com/questions/34052001/grouping-a-dataframe-and-applying-tuple)
positionalDf = df.groupby(np.arange(len(df.columns)) // 3, axis=1).apply(lambda x: pd.Series([tuple(i) for i in x.values]))

# stable_points = df.iloc[50:51] # Review Change to range at which it is stable rigid
# stable_points.apply(lambda x: x.mean(), axis=1)
# final_points = df.iloc[70:71] # Review Change to range at which we want to compare to
# final_points.apply(lambda x: x.mean(), axis=1)
# stable_points.index = range(1)
# final_points.index = range(1)
# print(stable_points)

# # Extract difference in position between each robot, at stable and final positions
# # print(stable_points.groupby(np.arange(len(stable_points.columns)) // 3, axis=1).groups)
# initial_separations = getSeparations(stable_points)
# final_separations = getSeparations(final_points)
# initial_separations

# TODO
# Get change in these separations,
#  by getting difference b/w all separations and
#  then combining, through an average across all subtraction results

In [ ]:
def plotAtTime(t):
    # get single row of positions
    positions_start = positionalDf[t:t+1].to_numpy().tolist()[0]
    # plot 2d (ignore z vals) (https://stackoverflow.com/questions/18458734/how-do-i-plot-list-of-tuples-in-python)
    plt.scatter(*list(zip(*positions_start))[:-1])
    plt.show()

In [ ]:
positionalDf.shape

In [ ]:
plotAtTime(0)

In [ ]:
plotAtTime(10000)

In [ ]:
plotAtTime(12000)

In [ ]:
plotAtTime(15000)

In [ ]:
plotAtTime(16000)

In [ ]:
plotAtTime(18000)

In [ ]:
plotAtTime(21980)

In [ ]:
fig = plt.figure()
axis = plt.axes(xlim =(-6, 15),  
                ylim =(-6, 8))
# ax.set(xlim=(-3, 3), ylim=(-1, 1))
plt.style.use("ggplot")

positions_start = positionalDf[0:0+1].to_numpy().tolist()[0]
sheep_pos_start = positions_start[:300]
dog_pos_start = positions_start[300:]
sheep_scat = axis.scatter(*( list(zip(*sheep_pos_start))[:-1] ), color='red')
dog_scat = axis.scatter(*( list(zip(*sheep_pos_start))[:-1] ), color='green')

def animate(t):
    positions = positionalDf[t:t+1].to_numpy().tolist()[0]
    positions = [x[0:2] for x in positions]
    sheep_pos_start = positions[:300]
    dog_pos_start = positions[300:]
    sheep_scat.set_offsets(sheep_pos_start)
    dog_scat.set_offsets(dog_pos_start)
    fig.suptitle(t, fontsize=12)

positions_over_time = animation.FuncAnimation(fig, animate, interval=5, frames=50)#21980)
# plt.draw()
# plt.show()
positions_over_time.save('positions.mp4')

In [ ]:
# Trying to follow https://stackoverflow.com/questions/39487901/interactive-animated-scatter-plotting-with-matplotlib

fig = plt.figure()  
axis = plt.axes(xlim =(-50, 50),  
                ylim =(-50, 50))
start_time = 0
end_time = 3

plt.ion()
plt.show()

for t in range(int(start_time), int(end_time), 1):
    positions = positionalDf[t:t+1].to_numpy().tolist()[0]
    (x,y) = list(zip(*positions))[:-1]
    axis.scatter(x, y)
    plt.pause(1.0)

In [ ]:
fig, ax = plt.subplots(1,1)
plt.subplots_adjust(bottom=0.25)
sax = plt.axes([0.25, 0.1, 0.65, 0.03])
start_time = 0
end_time = 3
slide = Slider(sax, 'time', start_time, end_time, valinit=start_time)

#Initial plot
positions_start = positionalDf[0:21980+1].to_numpy().tolist()[0]
s, = ax.plot(*( list(zip(*positions_start))[:-1] ))

def update(t):
    positions = positionalDf[t:t+1].to_numpy().tolist()[0]
    (x,y) = list(zip(*positions))[:-1]

    #Update data based on slider
    s.set_xdata(x)
    s.set_ydata(y)

    #Reset axis limits
    ax.set_xlim([x.min()*0.9, x.max()*1.1])
    ax.set_ylim([y.min()*0.9, y.max()*1.1])

    fig.canvas.draw()

slide.on_changed(update)
plt.show()